<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Получение-данных" data-toc-modified-id="Получение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Получение данных</a></span></li></ul></li><li><span><a href="#Исследование-данных" data-toc-modified-id="Исследование-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование данных</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных-для-обучения" data-toc-modified-id="Подготовка-данных-для-обучения-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка данных для обучения</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-dummy" data-toc-modified-id="Модель-dummy-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель dummy</a></span></li><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-RandomForestClassifier" data-toc-modified-id="Модель-RandomForestClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель RandomForestClassifier</a></span></li><li><span><a href="#Модель-CatBoostClassifier" data-toc-modified-id="Модель-CatBoostClassifier-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Модель CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Дообучение-и-тестирование-лучшей-модели" data-toc-modified-id="Дообучение-и-тестирование-лучшей-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Дообучение и тестирование лучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [30]:
!pip install catboost

In [31]:
import math
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

In [32]:
RANDOM_STATE = 18416996

### Получение данных

In [33]:
try:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
except:
    toxic_comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

## Исследование данных

In [34]:
toxic_comments.info()

Данные чисты и без пропусков. Однако их слишком уж много. Для первого теста стоит взять только часть данных для оценки моделей и ускорения исследования.

In [35]:
print(f"Максимальная длинна комментария: {max(toxic_comments['text'].str.len())}")

Комментарии с такой длинной не уложатся ни в одну модель. В таких случаях следует разделить комментарий на токенизированные части и дать оценку негативности для каждного куска. НО, мы просто их удалим. Укоротим датасет до данных с длинной до 1000 символов (должно уложиться в 512 положенных токенов). Затем возьмем сэмпл из скажем 10 тыс. строк, этого хватит и на трейн и на тест.

Проверим дисбаланс классов.

In [36]:
# Количество уникальных значений в столбце класса
num_classes = len(toxic_comments[toxic_comments['toxic'] == 1])
# Размер фрейма данных
num_samples = len(toxic_comments)
# Вычисление дисбаланса классов
imbalance_ratio = num_classes / num_samples
f'{imbalance_ratio: %}'

' 10.161213%'

Наблюдается сильный дисбаланс классов, учтем это при разделении данных.

Теперь досконально изучим каждый токсичный комментарий, вчитаемся в каждое слово, чтобы понять проблематику и уловить тончайшие зависимости для грамотного построения модели.

In [37]:
toxic_comments.query('toxic==1').head(20)

,Unnamed: 0,text,toxic
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,16,"Bye! \n\nDon't look, come or think of comming ...",1
42,42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
44,44,I'm Sorry \n\nI'm sorry I screwed around with ...,1
51,51,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,1
55,55,Stupid peace of shit stop deleting my stuff as...,1
56,56,=Tony Sidaway is obviously a fistfuckee. He lo...,1
58,58,My Band Page's deletion. You thought I was gon...,1


Вывод: Данные чистые, но их слишком много. Урежем датасет в несколько раз для комфортной работы с ним. Также наблюдается сильный дисбаланс классов в сторону отсутсвия токсичности комментария. Это следует учесть при разбиении данных на тест и трейн. Также некоторые комментарии излишне длинные и их придется вырезать из общей массы.

Выберем уже предобученные токенизатор и модель BERT из популярной библиотеки distilbert.

### Подготовка данных для обучения

In [38]:
toxic_comments = toxic_comments[toxic_comments['text'].str.len() < 1000]
samples = 7000# round(len(toxic_comments) / 8)
toxic_comments = toxic_comments.sample(random_state=RANDOM_STATE, n=samples)
toxic_comments.info()

С таким количеством данных будет приятнее иметь дело!

In [39]:
# model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')
# 
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)
model_name = "unitary/toxic-bert"
model = transformers.AutoModel.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Для обучения токенизируем текста в векторы с одинаковой длинной. Но естественно длинна не может превышать 512 токенов. Также зададим маску для отсеивания нулевых значений.

In [40]:
tokenized = toxic_comments['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = max(len(i) for i in tokenized.values) # 512

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values if len(i) <= max_len])

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(7000, 404)

Создадим эмбеддинги для обучения моделей

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
device

device(type='cpu')

In [42]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(math.ceil(padded.shape[0] / batch_size))):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/70 [00:00<?, ?it/s]

Далее сформируем массивы valid и train для тестирования и обучения моделей с учетом явного дисбаланса.

In [ ]:
features, features_test = train_test_split(np.concatenate(embeddings), test_size=.25, random_state=RANDOM_STATE)
target, target_test = train_test_split(toxic_comments['toxic'], test_size=.25, random_state=RANDOM_STATE)

In [45]:

X_train, X_valid, y_train, y_valid = train_test_split(features, target, stratify=target, random_state=RANDOM_STATE)

## Обучение

### Модель dummy

In [46]:
print(f"Результат dummy модели {f1_score(np.ones(len(X_valid)), y_valid)}")

### Модель LogisticRegression

In [47]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict_proba(X_valid)[:, 1]
print(f"Результат модели логистической регрессии {f1_score(np.round(predictions), y_valid)}")

### Модель RandomForestClassifier

In [48]:
best_rf_model1_dt_model, best_rf_model1_predicted_valid = None, None
best_rf_model1_result, best_rf_model1_est, best_rf_model1_depth = 10000, 0, 0

In [49]:
for n_estimators in tqdm(range(30, 131, 20)):
    for depth in range(6, 15, 4):
        rf_model1 = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=depth, n_estimators=n_estimators)
        rf_model1.fit(X_train, y_train)
        rf_model1_predictions_valid = rf_model1.predict_proba(X_valid)[:, 1]
        result = f1_score(np.round(rf_model1_predictions_valid), y_valid)
        if result < best_rf_model1_result:
            best_rf_model1 = rf_model1
            best_rf_model1_result = result
            best_rf_model1_est = n_estimators
            best_rf_model1_depth = depth
            best_rf_model1_predicted_valid = rf_model1_predictions_valid

In [50]:
print(f"F1 наилучшей модели случайного леса на валидационной выборке: {best_rf_model1_result}")

### Модель CatBoostClassifier

In [51]:
params = {
    'loss_function': 'Logloss',  # objective function Logloss
    'eval_metric': 'F1',  # metric
    'verbose': 200,  # output to stdout info about training process every 200 iterations
    'random_seed': RANDOM_STATE}
cbc_1 = CatBoostClassifier(**params)
cbc_1.fit(
    X_train, y_train,
    # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
    eval_set=(X_valid, y_valid),  # data to validate on
    use_best_model=True,  # True if we don't want to save trees created after iteration with the best validation score
    plot=True
    # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [52]:
predictions = cbc_1.predict_proba(X_valid)[:, 1]
print(f1_score(np.round(predictions), y_valid))

## Дообучение и тестирование лучшей модели

In [53]:
X = np.concatenate([X_train, X_valid])
y = np.concatenate([y_train, y_valid])

In [54]:
lr.fit(X, y)

LogisticRegression()

In [56]:
final_predictions = lr.predict_proba(features_test)[:, 1]
f1_score(np.round(final_predictions), target_test)

0.9325513196480938

## Выводы

С помощью полученных от заказчика данных был разработан инструмент для определения токсичных комментариев.
Для это была использована языковая модель toxic_Bert из открытого доступа.
С помощью данной модели текст сообщений был разбит на параметры, с помощью которых можно обучать модели машинного обучения.
Для валидации были обучены три модели:
- Модель-предсказатель единиц!
    Результат метрики F1 модели-провидца 0.19 - можем отталкиваться от этого значения для адекватной оценки остальных моделей
- Модель логистической регрессии LogisticRegression
    Результат метрики F1 модели логистической регрессии 0.96
- Модель случайного леса деревьев решений RandomForestClassifier
    Результат метрики F1 лучшей модели случайного леса 0.96
- Модель градиентного бустинга деревьев решений CatBoostClassifier
    Результат метрики F1 лучшей итерации CatBoost 0.93

В итоге вышло, что самая легковесная модель LogisticRegression оказалась самой точной в данной задаче.
Для тестирования модель была дообучена на валидационных данных и показала итоговую метрику F1 = 0.93 на тестовых данных
Данная метрика показывает, что если модель предобучена и правильно подобрана, то данных для обучения требуется в разы меньше!